# Exercise Sheet \# 7 - Interfacing Python with SQL databases

In this exercises, you are asked to design python code which can interact with a SQLite database. We will use again the [SQL database](http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip) from Exercise Sheet \#6. 

## Exercise 1

Write a python script, which covers questions 1.5 to 1.9 from Exercise Sheet \#5, using the SQLalchemy API.

In [7]:
from sqlalchemy import create_engine, distinct, func
from sqlalchemy import MetaData, Table
import sqlalchemy as db

engine = create_engine('sqlite:///chinook.db')
connection = engine.connect()
metadata = MetaData()

#1.5
tracks = Table('tracks', metadata, autoload_with=engine)
query = db.select(tracks.c.AlbumId, func.count(tracks.c.TrackId)).group_by(tracks.c.AlbumId)
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
for t in result_set:
    print(f'AlbumId {t[0]}, tracks: {t[1]}')

AlbumId 1, tracks: 10
AlbumId 2, tracks: 1
AlbumId 3, tracks: 3
AlbumId 4, tracks: 8
AlbumId 5, tracks: 15
AlbumId 6, tracks: 13
AlbumId 7, tracks: 12
AlbumId 8, tracks: 14
AlbumId 9, tracks: 8
AlbumId 10, tracks: 14
AlbumId 11, tracks: 12
AlbumId 12, tracks: 12
AlbumId 13, tracks: 8
AlbumId 14, tracks: 13
AlbumId 15, tracks: 5
AlbumId 16, tracks: 7
AlbumId 17, tracks: 10
AlbumId 18, tracks: 17
AlbumId 19, tracks: 11
AlbumId 20, tracks: 11
AlbumId 21, tracks: 18
AlbumId 22, tracks: 3
AlbumId 23, tracks: 34
AlbumId 24, tracks: 23
AlbumId 25, tracks: 13
AlbumId 26, tracks: 17
AlbumId 27, tracks: 14
AlbumId 28, tracks: 10
AlbumId 29, tracks: 14
AlbumId 30, tracks: 14
AlbumId 31, tracks: 9
AlbumId 32, tracks: 14
AlbumId 33, tracks: 17
AlbumId 34, tracks: 17
AlbumId 35, tracks: 11
AlbumId 36, tracks: 17
AlbumId 37, tracks: 20
AlbumId 38, tracks: 12
AlbumId 39, tracks: 21
AlbumId 40, tracks: 12
AlbumId 41, tracks: 14
AlbumId 42, tracks: 14
AlbumId 43, tracks: 7
AlbumId 44, tracks: 6
AlbumId 

In [8]:
#1.6: average duration of songs (milliseconds)
query = db.select(func.avg(tracks.c.Milliseconds))
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
print("Average duration of songs:", result_set[0][0])

Average duration of songs: 393599.2121039109


In [9]:
#1.7: add Bono to artist table
artists = Table('artists', metadata, autoload_with=engine)
#print(artists._columns)
query = db.insert(artists).values(ArtistId = 500, Name='Bono')
print(query.compile().params)

{'ArtistId': 500, 'Name': 'Bono'}


In [11]:
# 1.8: list artists that do not appear in albums table
albums = Table('albums', metadata, autoload_with = engine)
#query = db.select(artists.c.ArtistId).join(albums, isouter = True).where(albums.c.AlbumId==None)
subquery = db.select(distinct(albums.c.ArtistId))
query = db.select(artists.c.ArtistId).where(artists.c.ArtistId.notin_(subquery))
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
print(result_set) ## not sure this is correct 

## delete:

query = db.delete(artists).where(artists.c.ArtistId.notin_(subquery))

[]


In [12]:
#1.9. remove artists whose name starts with N:
query = db.delete(artists).where(artists.c.Name.like('N%'))
print(query)

DELETE FROM artists WHERE artists."Name" LIKE :Name_1


## Exercise 2
Extend your program so that it also covers questions 2.5, 2.6 and 2.8 from previous Exercise Sheet.

In [13]:
# 2.5.How many albums and songs from Philip Glass Ensemble
query = db.select(albums.c.Title, func.count(tracks.c.TrackId)).where(db.and_(albums.c.AlbumId == tracks.c.AlbumId,
                                                                                            albums.c.ArtistId == artists.c.ArtistId,
                                                                                            artists.c.Name == 'Philip Glass Ensemble')).group_by(albums.c.Title)
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
print(result_set)

[('Koyaanisqatsi (Soundtrack from the Motion Picture)', 1)]


In [14]:
#2.6 which artist plays longest song 

subquery = db.select(func.max(tracks.c.Milliseconds))
query = db.select(artists.c.Name).select_from(artists).join(albums).join(tracks).where(tracks.c.Milliseconds == subquery)

result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
print(result_set[0][0]) 

Battlestar Galactica


/var/folders/67/_dh47txs4m56s30j9cjp7w4w0000gn/T/ipykernel_1890/3664647926.py:4: SAWarning: implicitly coercing SELECT object to scalar subquery; please use the .scalar_subquery() method to produce a scalar subquery.
  query = db.select(artists.c.Name).select_from(artists).join(albums).join(tracks).where(tracks.c.Milliseconds == subquery)


In [ ]:
# 2.8. genres without a song
genres = Table('genres', metadata, autoload_with = engine)

subquery = db.select(tracks.c.GenreId)
query = db.select(genres.c.Name).where(genres.c.GenreId.notin_(subquery))
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
print(result_set)
#print(query)